In [1]:
import pandas as pd
import glob
import os

%cd ~/openfold

/net/pulsar/home/koes/jok120/openfold


In [2]:
exp_dir = "/net/pulsar/home/koes/jok120/openfold/jk_research/evaluations/experiment_directory.csv"
eval_jobs = "/net/pulsar/home/koes/jok120/openfold/jk_research/evaluations/230418/eval_jobs.csv"

# Load and join the experiment directory and evaluation job directory on exp_name
exp_dir = pd.read_csv(exp_dir)
eval_jobs = pd.read_csv(eval_jobs)
df = eval_jobs.merge(exp_dir, on="exp_name", how="left")

In [3]:
# add a column for the new_exp_name that combines the exp_suffix and exp_name with "-"
df['exp_suffix'].fillna('eval_sx', inplace=True)
df["new_exp_name"] = df["exp_name"] + "-" + df["exp_suffix"]
df 

,exp_suffix,exp_name,eval_me,wandb_id,location,notes,new_exp_name
0,eval,initial_training,True,NaN,NaN,NaN,initial_training-eval
1,eval_s945,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_s945
2,eval_sx,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_sx
3,eval_sx,baseline-scnmin-omm-00c,True,1w4vxghv,crc,Resumed baseline-scnmin-omm-00b,baseline-scnmin-omm-00c-eval_sx
4,eval_sx,xfer-scnmin-omm-01,True,c2bs3y0h,g019,Resume baseline-scnmin-noomm-00 at about 1k st...,xfer-scnmin-omm-01-eval_sx
5,eval_sx,xfer-scnmin-omm-02,True,2adahxb4,crc,"Resume baseline at 3.8k steps, turning on omm.",xfer-scnmin-omm-02-eval_sx
6,eval_sx,xfer-scnmin-omm-03,True,25qsm73w,crc,"Same as xfer-scnin-omm-01, but modifies OMM to...",xfer-scnmin-omm-03-eval_sx
7,eval_sx,xfer-scnmin-omm-04,False,22otzx3o,crc,"Same as xfer-scnmin-omm-01, but scales omm by ...",xfer-scnmin-omm-04-eval_sx
8,eval_sx,xfer-scnmin-omm-05,False,19qb4yhu,crc,"Same as xfer-scnmin-omm-01, but scales omm by ...",xfer-scnmin-omm-05-eval_sx


In [4]:
# Filter out rows that have location == crc
df = df[df["location"] == "g019"]
df

,exp_suffix,exp_name,eval_me,wandb_id,location,notes,new_exp_name
1,eval_s945,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_s945
2,eval_sx,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_sx
4,eval_sx,xfer-scnmin-omm-01,True,c2bs3y0h,g019,Resume baseline-scnmin-noomm-00 at about 1k st...,xfer-scnmin-omm-01-eval_sx


In [5]:
# We now need to determine the path to the checkpoint file for each experiment.
# The checkpoint file can be found at out/experiments/*/finetune-openfold-02/{wandb_id}/checkpoints
# We can use the wandb_id to find the correct experiment directory
# We can use the exp_suffix to determine which checkpoint (eval_s! means the last checkpoint),
# while eval_sN means the checkpoint at step N.
# Checkpoints are titled "{epoch}-{step}.ckpt".

# First, we need to determine the path to the experiment directory
# We can do this by using the wandb_id to find the correct experiment directory
# The experiment directory is the first directory in the out/experiments/*/finetune-openfold-02/{wandb_id} directory
# We can use the wandb_id to find the correct experiment directory

# Find the actual experiment directory
df["exp_dir"] = df["wandb_id"].apply(lambda x: f"out/experiments/*/finetune-openfold-02/{x}")
# Expand the glob
df["exp_dir"] = df["exp_dir"].apply(lambda x: glob.glob(x)[0])
df

/tmp/ipykernel_3652306/861607784.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["exp_dir"] = df["wandb_id"].apply(lambda x: f"out/experiments/*/finetune-openfold-02/{x}")
/tmp/ipykernel_3652306/861607784.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["exp_dir"] = df["exp_dir"].apply(lambda x: glob.glob(x)[0])


,exp_suffix,exp_name,eval_me,wandb_id,location,notes,new_exp_name,exp_dir
1,eval_s945,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_s945,out/experiments/230319/finetune-openfold-02/3l...
2,eval_sx,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_sx,out/experiments/230319/finetune-openfold-02/3l...
4,eval_sx,xfer-scnmin-omm-01,True,c2bs3y0h,g019,Resume baseline-scnmin-noomm-00 at about 1k st...,xfer-scnmin-omm-01-eval_sx,out/experiments/230407_xfer01/finetune-openfol...


In [6]:
# Next, we need to determine the path to the checkpoint file
# We can do this by using the exp_suffix to determine which checkpoint (eval_s! means the last checkpoint),
# while eval_sN means the checkpoint at step N.


def get_checkpoint_path(row):
    if row["exp_suffix"] == "eval_sx":
        # Find the filename with the highest epoch number
        files = os.listdir(f"{row['exp_dir']}/checkpoints")
        files = [f for f in files if f.endswith(".ckpt")]
        files = [f.split("-") for f in files]  # look at epoch and step
        files = [(int(f[0]), int(f[1].split(".")[0])) for f in files]
        files = sorted(files, key=lambda x: (x[0], x[1]))  # find the highest epoch/step
        last_file = files[-1]
        return f"{row['exp_dir']}/checkpoints/{last_file[0]}-{last_file[1]}.ckpt"
    else:
        step = row["exp_suffix"].split("_")[1][1:]
        #  Find the checkpoint with the closest step number
        files = os.listdir(f"{row['exp_dir']}/checkpoints")
        files = [f for f in files if f.endswith(".ckpt")]
        files = [f.split("-") for f in files]
        files = [(int(f[0]), int(f[1].split(".")[0])) for f in files]
        # Get the closest step number
        closest_step = min(files, key=lambda x: abs(x[1] - int(step)))
        # Return the path to the closest checkpoint
        return f"{row['exp_dir']}/checkpoints/{closest_step[0]}-{closest_step[1]}.ckpt"

df["checkpoint_path"] = df.apply(get_checkpoint_path, axis=1)
df


/tmp/ipykernel_3652306/1533968007.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["checkpoint_path"] = df.apply(get_checkpoint_path, axis=1)


,exp_suffix,exp_name,eval_me,wandb_id,location,notes,new_exp_name,exp_dir,checkpoint_path
1,eval_s945,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_s945,out/experiments/230319/finetune-openfold-02/3l...,out/experiments/230319/finetune-openfold-02/3l...
2,eval_sx,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline-scnmin-noomm-00-eval_sx,out/experiments/230319/finetune-openfold-02/3l...,out/experiments/230319/finetune-openfold-02/3l...
4,eval_sx,xfer-scnmin-omm-01,True,c2bs3y0h,g019,Resume baseline-scnmin-noomm-00 at about 1k st...,xfer-scnmin-omm-01-eval_sx,out/experiments/230407_xfer01/finetune-openfol...,out/experiments/230407_xfer01/finetune-openfol...


In [7]:
# Define slurm preamble
location = 'g019'
if location == "crc":
    preamble = """#SBATCH --job-name=evalval
#SBATCH --nodes=1
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=4
#SBATCH --partition=a100
#SBATCH --qos=short
#SBATCH --time=04:00:00
#SBATCH --cluster=gpu
#SBATCH --output="/ihome/dkoes/jok120/openfold/out/%A_%6a.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jok120@pitt.edu"""

elif location == "g019":
    preamble = """#SBATCH --job-name=xfer01
#SBATCH --nodes=1
#SBATCH --gres=gpu:4
#SBATCH --nodelist=g019
#SBATCH --ntasks-per-node=64
#SBATCH --time=14-00:00:00
#SBATCH --partition=dept_gpu
#SBATCH --output="/net/pulsar/home/koes/jok120/openfold/out/%A_%6a.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jok120@pitt.edu"""

In [8]:
# Define most-likely static variables
TRAIN_STRUCTURES_DIR="data/train_structs/scnmin_structs/"
ALIGNMENTS_DIR="data/alignments/scnmin_alignments/"
TEMPLATE_STRUCTURES_DIR="data/template_structs/roda_pdbs_snapshotted_flattened_do_not_overwrite/"
TRAIN_CACHE="data/caches/chain_data_cache_rodasnapshot_clustered.json"
TEMPLATE_CACHE="data/caches/mmcif_cache_rodasnapshot.json"

# Define variables that will be different for each job
EXPERIMENT_NAME="this_is_a_test"
OUTDIR=f"out/evaluations/{EXPERIMENT_NAME}"
CHECKPOINT="openfold/resources/openfold_params/initial_training.pt"
VALIDATION_STRUCTURES_DIR="data/validation/cameo/20220116/minimized/data_dir"
VALIDATION_ALIGNMENTS_DIR="data/validation/cameo/20220116/minimized/alignments"
# VALIDATION_STRUCTURES_DIR="data/validation/cameo/20220116/test/data_dir"
# VALIDATION_ALIGNMENTS_DIR="data/validation/cameo/20220116/test/alignments"

In [9]:
# read the string from subjob_skeleton.slurm into slurm_str
with open("jk_research/evaluations/230418/subjob_skeleton.slurm", "r") as f:
    slurm_str = f.read()

In [10]:
# Iterate through each row in the dataframe and create a slurm file for each row
# Fill in the EXPERIMENT_NAME, OUTDIR, and CHECKPOINT variables with data from the dataframe
# also create the subjob directory if it doesn't exist
if not os.path.exists("jk_research/evaluations/230418/subjobs"):
    os.mkdir("jk_research/evaluations/230418/subjobs")
for index, row in df.iterrows():
    print(f"Creating slurm file for {row['new_exp_name']}")
    # Create the slurm file
    # For the file and the output directory, we need to replace the "-" with "_"
    # because "-" is not allowed in the file name
    slurm_file = f"jk_research/evaluations/230418/subjobs/{row['new_exp_name'].replace('-', '_')}.slurm"
    with open(slurm_file, "w") as f:
        f.write(slurm_str.format(
            preamble=preamble,
            EXPERIMENT_NAME=row["new_exp_name"],
            OUTDIR=f"out/evaluations/{row['new_exp_name'].replace('-', '_')}",
            CHECKPOINT=row["checkpoint_path"],
            VALIDATION_STRUCTURES_DIR=VALIDATION_STRUCTURES_DIR,
            VALIDATION_ALIGNMENTS_DIR=VALIDATION_ALIGNMENTS_DIR,
            TRAIN_STRUCTURES_DIR=TRAIN_STRUCTURES_DIR,
            ALIGNMENTS_DIR=ALIGNMENTS_DIR,
            TEMPLATE_STRUCTURES_DIR=TEMPLATE_STRUCTURES_DIR,
            TRAIN_CACHE=TRAIN_CACHE,
            TEMPLATE_CACHE=TEMPLATE_CACHE,
        ))

Creating slurm file for baseline-scnmin-noomm-00-eval_s945
Creating slurm file for baseline-scnmin-noomm-00-eval_sx
Creating slurm file for xfer-scnmin-omm-01-eval_sx


In [11]:
print(slurm_str.format(**locals()))

#!/bin/bash
#SBATCH --job-name=xfer01
#SBATCH --nodes=1
#SBATCH --gres=gpu:4
#SBATCH --nodelist=g019
#SBATCH --ntasks-per-node=64
#SBATCH --time=14-00:00:00
#SBATCH --partition=dept_gpu
#SBATCH --output="/net/pulsar/home/koes/jok120/openfold/out/%A_%6a.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jok120@pitt.edu


############################
##       Description      ##
############################
echo "Running job ${SLURM_JOB_ID} with ${SLURM_NTASKS} workers on node ${SLURMD_NODENAME}."
echo "Experiment: this_is_a_test"

############################
##       Environment      ##
############################
source scripts/activate_conda_env.sh
module load cuda/11.3.0
module load gcc/8.2.0


############################
##     Array Job Exec.    ##
############################
mkdir -p out/evaluations/this_is_a_test


./train_openfold.py data/train_structs/scnmin_structs/ data/alignments/scnmin_alignments/ data/template_structs/roda_pdbs_snapshotted_flattened_do_not_overwrite/ out